In [ ]:
import tensorflow as tf
from keras.applications import MobileNetV3Small

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set hyperparameters
input_shape = (224, 224, 3)
num_classes = 4
pretrain_epochs = 10
fine_tune_epochs = 10
batch_size = 32

In [ ]:
# Load unlabeled brain tumor data
unlabeled_data = tf.keras.preprocessing.image_dataset_from_directory(
    "../content/drive/MyDrive/brain_tumor_unsupervised/unsupervised",
    image_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size
)

# Load labeled brain tumor classification data
training_data = tf.keras.preprocessing.image_dataset_from_directory(
    "../content/drive/MyDrive/brain_tumor_unsupervised/training",
    image_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    'path/to/test/data',
    image_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size
)

##Unsupervised learning

In [ ]:
# Define self-supervised learning model
pretrain_model = MobileNetV3Small(
    input_shape=input_shape,
    include_top=False,
    weights=None
)

# Add self-supervised learning head
pretrain_model = tf.keras.Sequential([
    pretrain_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(4, activation='sigmoid')
])

# Compile self-supervised learning model
pretrain_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# Train self-supervised learning model on unlabeled data
pretrain_model.fit(unlabeled_data, epochs=pretrain_epochs)

##Supervised learning

In [ ]:
# Freeze layers of pre-trained model and add classification head
base_model = MobileNetV3Small(
    input_shape=input_shape,
    include_top=False,
    weights=None
)
for layer in base_model.layers:
    layer.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile fine-tuned model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# Fine-tune model on labeled brain tumor classification data
model.fit(training_data, epochs=fine_tune_epochs, test_data = test_data)

In [ ]:
# Evaluate model on test data
test_loss, test_acc = model.evaluate(test_data)
print('Test accuracy:', test_acc)